# Sales & Demand Forecasting

This notebook extends the pizza sales analysis to include predictive modeling and forecasting capabilities. It builds upon the data structures and insights from `Main.ipynb` and `Inventory_Tracking.ipynb` to predict future sales patterns and ingredient demand.

In [ ]:
# Cell 1 — Purpose and rationale
# Purpose: Import third‑party libraries and standard modules required downstream. Fit a regression‑based forecasting model and generate predictions. Evaluate predictive performance using standard regression metrics. Visualize distributions, trends, and seasonality for validation and communication.
# Inputs: Expects numeric feature matrix X and target vector y after preprocessing.
# Outputs: No side‑effects beyond state changes of in‑notebook variables.
# Key operations:
#  • pandas for tabular data manipulation
#  • numpy for vectorized numerical routines
#  • matplotlib for plotting
#  • seaborn for statistical visualization
#  • scipy.stats for statistical utilities
#  • datetime for date/time handling
#  • collections.defaultdict for aggregation helpers
#  • scikit‑learn components: LinearRegression (ordinary least squares), PolynomialFeatures to expand basis for non‑linear trends, regression metrics (MAE/MSE/RMSE/R²)
#  • Expand scalar time index into polynomial basis to capture curvature (risk of overfitting at high degree).
#  • Fit OLS on engineered features; check residual patterns to ensure assumptions are reasonable.
#  • Compute MAE/MSE; RMSE = sqrt(MSE); lower is better. Optionally inspect R² for explained variance.
#  • Generate plots to validate stationarity/seasonality, anomalies, and model fit.
# Assumptions/Caveats: Degree selection should be validated via cross‑validation to avoid high‑variance fits. OLS assumes linearity in parameters and limited multicollinearity; feature scaling may help stability.
# --- End of documentation block ---

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

# Statistical and ML libraries for forecasting
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import PolynomialFeatures

# Set visualization style
plt.style.use('seaborn-v0_8')
sns.set_palette('husl')

## Data Loading and Preparation

Load the pizza sales data and prepare it for time series analysis.

In [ ]:
# Cell 2 — Purpose and rationale
# Purpose: Load raw data into a pandas DataFrame for subsequent preparation and modeling. Normalize temporal fields and derive calendar/time features for seasonality analysis.
# Inputs: Reads Excel file path(s) specified in the code into a DataFrame. Consumes a pandas DataFrame produced in prior cells.
# Outputs: Produces in‑memory DataFrame(s)/array(s) assigned to variables for later steps. Emits textual summaries to stdout for quick verification.
# Key operations:
#  • Read raw data; ensure correct dtypes and parse dates if applicable.
#  • Convert to pandas datetime and derive features (hour, weekday, week, month, dayofyear).
# Assumptions/Caveats: Review intermediate outputs to ensure data and features conform to expectations.
# --- End of documentation block ---

# Load data
df = pd.read_excel('Data Model - Pizza Sales.xlsx')
df['order_date'] = pd.to_datetime(df['order_date'])
df['order_time'] = pd.to_datetime(df['order_time'], format='%H:%M:%S').dt.time
df['hour'] = pd.to_datetime(df['order_time'], format='%H:%M:%S').dt.hour
df['weekday'] = df['order_date'].dt.day_name()
df['day_of_year'] = df['order_date'].dt.dayofyear
df['week_of_year'] = df['order_date'].dt.isocalendar().week
df['month'] = df['order_date'].dt.month

print(f"Data loaded: {len(df)} orders from {df['order_date'].min()} to {df['order_date'].max()}")
print(f"Total revenue: €{df['total_price'].sum():,.2f}")
print(f"Total pizzas sold: {df['quantity'].sum():,}")

Data loaded: 48620 orders from 2015-01-01 00:00:00 to 2015-12-31 00:00:00
Total revenue: €817,860.05
Total pizzas sold: 49,574


## Time Series Data Preparation

Create daily aggregated data for forecasting models.

In [ ]:
# Cell 3 — Purpose and rationale
# Purpose: Normalize temporal fields and derive calendar/time features for seasonality analysis. Aggregate event‑level records to daily granularity and compute descriptive metrics.
# Inputs: Consumes a pandas DataFrame produced in prior cells.
# Outputs: Produces in‑memory DataFrame(s)/array(s) assigned to variables for later steps. Emits textual summaries to stdout for quick verification.
# Key operations:
#  • Convert to pandas datetime and derive features (hour, weekday, week, month, dayofyear).
#  • Group by calendar date; aggregate sales count/revenue/orders; compute AOV and similar ratios if defined.
# Assumptions/Caveats: Index manipulation aligns with grouping/merging operations; ensure monotonic date index for time‑series tasks.
# --- End of documentation block ---

# Create daily aggregated data
daily_sales = df.groupby('order_date').agg({
    'quantity': 'sum',
    'total_price': 'sum',
    'order_id': 'nunique'  # number of unique orders
}).reset_index()

daily_sales.columns = ['date', 'pizzas_sold', 'revenue', 'orders_count']
daily_sales['avg_order_value'] = daily_sales['revenue'] / daily_sales['orders_count']
daily_sales['avg_pizzas_per_order'] = daily_sales['pizzas_sold'] / daily_sales['orders_count']

# Add time features for modeling
daily_sales['weekday'] = daily_sales['date'].dt.day_name()
daily_sales['day_of_year'] = daily_sales['date'].dt.dayofyear
daily_sales['week_of_year'] = daily_sales['date'].dt.isocalendar().week
daily_sales['month'] = daily_sales['date'].dt.month
daily_sales['is_weekend'] = daily_sales['date'].dt.weekday >= 5

print(f"Daily sales data prepared: {len(daily_sales)} days")
daily_sales.head()

Daily sales data prepared: 358 days


,date,pizzas_sold,revenue,orders_count,avg_order_value,avg_pizzas_per_order,weekday,day_of_year,week_of_year,month,is_weekend
0,2015-01-01,162,2713.85,69,39.331159,2.347826,Thursday,1,1,1,False
1,2015-01-02,165,2731.90,67,40.774627,2.462687,Friday,2,1,1,False
2,2015-01-03,158,2662.40,66,40.339394,2.393939,Saturday,3,1,1,True
3,2015-01-04,106,1755.45,52,33.758654,2.038462,Sunday,4,1,1,True
4,2015-01-05,125,2065.95,54,38.258333,2.314815,Monday,5,2,1,False


## Model Training and Setup

Train the forecasting models and prepare data structures.

In [ ]:
# Cell 4 — Purpose and rationale
# Purpose: Define a deterministic feature engineering pipeline for forecasting inputs.
# Inputs: Accepts function arguments as declared; ensure types align with usage (e.g., pandas Timestamp, ndarray).
# Outputs: Returns computed objects explicitly from function(s). Emits textual summaries to stdout for quick verification.
# Key operations:
#  • Construct deterministic feature set; maintain identical logic for train and inference.
#  • Encode periodicity with sine/cosine transforms: sin(2π·t/P), cos(2π·t/P) for period P (e.g., 7, 52, 365).
#  • Include calendar flags (weekday number, weekend indicator).
# Assumptions/Caveats: Review intermediate outputs to ensure data and features conform to expectations.
# --- End of documentation block ---

def create_features_for_forecasting(data):
    """Create numerical features for forecasting models"""
    features_df = data.copy()
    
    # Encode categorical variables
    weekday_mapping = {'Monday': 1, 'Tuesday': 2, 'Wednesday': 3, 'Thursday': 4, 
                      'Friday': 5, 'Saturday': 6, 'Sunday': 7}
    features_df['weekday_num'] = features_df['weekday'].map(weekday_mapping)
    
    # Cyclical encoding for seasonal patterns
    features_df['day_sin'] = np.sin(2 * np.pi * features_df['day_of_year'] / 365)
    features_df['day_cos'] = np.cos(2 * np.pi * features_df['day_of_year'] / 365)
    features_df['week_sin'] = np.sin(2 * np.pi * features_df['weekday_num'] / 7)
    features_df['week_cos'] = np.cos(2 * np.pi * features_df['weekday_num'] / 7)
    
    return features_df

# Prepare features
forecast_data = create_features_for_forecasting(daily_sales)
feature_cols = ['day_of_year', 'weekday_num', 'month', 'is_weekend', 
                'day_sin', 'day_cos', 'week_sin', 'week_cos']

# Split data (80% train, 20% test)
split_idx = int(len(forecast_data) * 0.8)
train_data = forecast_data.iloc[:split_idx]
test_data = forecast_data.iloc[split_idx:]

# Prepare features and targets
X_train = train_data[feature_cols]
X_test = test_data[feature_cols]
y_train_pizzas = train_data['pizzas_sold']
y_test_pizzas = test_data['pizzas_sold']
y_train_revenue = train_data['revenue']
y_test_revenue = test_data['revenue']

print(f"Training data: {len(train_data)} days")
print(f"Testing data: {len(test_data)} days")
print(f"Features: {feature_cols}")

Training data: 286 days
Testing data: 72 days
Features: ['day_of_year', 'weekday_num', 'month', 'is_weekend', 'day_sin', 'day_cos', 'week_sin', 'week_cos']


In [ ]:
# Cell 5 — Purpose and rationale
# Purpose: Fit a regression‑based forecasting model and generate predictions. Translate forecasted demand into bill‑of‑materials quantities via recipe composition.
# Inputs: Consumes a pandas DataFrame produced in prior cells. Expects numeric feature matrix X and target vector y after preprocessing.
# Outputs: Produces in‑memory DataFrame(s)/array(s) assigned to variables for later steps. Emits textual summaries to stdout for quick verification.
# Key operations:
#  • Expand scalar time index into polynomial basis to capture curvature (risk of overfitting at high degree).
#  • Fit OLS on engineered features; check residual patterns to ensure assumptions are reasonable.
#  • Model training on historical observations.
#  • Map forecasted pizza counts to ingredient quantities via per‑pizza recipe weights.
# Assumptions/Caveats: Index manipulation aligns with grouping/merging operations; ensure monotonic date index for time‑series tasks. Degree selection should be validated via cross‑validation to avoid high‑variance fits. OLS assumes linearity in parameters and limited multicollinearity; feature scaling may help stability.
# --- End of documentation block ---

# Train forecasting models
models = {}

# Linear regression for pizza demand
models['pizza_linear'] = LinearRegression()
models['pizza_linear'].fit(X_train, y_train_pizzas)

# Polynomial regression for pizza demand
poly_features = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly_features.fit_transform(X_train)
X_test_poly = poly_features.transform(X_test)
models['pizza_poly'] = LinearRegression()
models['pizza_poly'].fit(X_train_poly, y_train_pizzas)

# Linear regression for revenue
models['revenue_linear'] = LinearRegression()
models['revenue_linear'].fit(X_train, y_train_revenue)

print("✅ Models trained successfully!")

# Set up recipe data for ingredient predictions
df['ingredient_list'] = df['pizza_ingredients'].str.split(',').apply(lambda lst: [ing.strip() for ing in lst])
recipes_dict = (
    df[['pizza_name', 'ingredient_list']]
    .drop_duplicates('pizza_name')
    .set_index('pizza_name')['ingredient_list']
    .to_dict()
)
pizza_mix = df.groupby('pizza_name')['quantity'].sum() / df['quantity'].sum()

print(f"✅ Recipe data loaded: {len(recipes_dict)} pizza types")
print(f"✅ Pizza mix calculated for ingredient predictions")

✅ Models trained successfully!
✅ Recipe data loaded: 32 pizza types
✅ Pizza mix calculated for ingredient predictions


## Prediction Functions

In [ ]:
# Cell 6 — Purpose and rationale
# Purpose: Normalize temporal fields and derive calendar/time features for seasonality analysis. Provide a callable interface to forecast demand for arbitrary target dates. Translate forecasted demand into bill‑of‑materials quantities via recipe composition.
# Inputs: Consumes a pandas DataFrame produced in prior cells. Accepts function arguments as declared; ensure types align with usage (e.g., pandas Timestamp, ndarray). Expects numeric feature matrix X and target vector y after preprocessing.
# Outputs: Returns computed objects explicitly from function(s). Emits textual summaries to stdout for quick verification.
# Key operations:
#  • Convert to pandas datetime and derive features (hour, weekday, week, month, dayofyear).
#  • Inference using consistent feature pipeline for target horizon(s).
#  • Function builds features for provided date(s) and returns point forecasts.
#  • Map forecasted pizza counts to ingredient quantities via per‑pizza recipe weights.
# Assumptions/Caveats: Review intermediate outputs to ensure data and features conform to expectations.
# --- End of documentation block ---

def predict_future_demand(start_date, end_date, model_type='linear'):
    """Predict pizza demand and revenue for future dates"""
    if isinstance(start_date, str):
        start_date = pd.to_datetime(start_date)
    if isinstance(end_date, str):
        end_date = pd.to_datetime(end_date)
    
    # Create future date range
    future_dates = pd.date_range(start=start_date, end=end_date, freq='D')
    
    # Create future features
    future_df = pd.DataFrame({
        'date': future_dates,
        'day_of_year': future_dates.dayofyear,
        'weekday': future_dates.day_name(),
        'month': future_dates.month,
        'is_weekend': future_dates.weekday >= 5
    })
    
    future_df = create_features_for_forecasting(future_df)
    X_future = future_df[feature_cols]
    
    # Make predictions
    if model_type == 'polynomial':
        X_future_transformed = poly_features.transform(X_future)
        future_df['predicted_pizzas'] = models['pizza_poly'].predict(X_future_transformed)
    else:
        future_df['predicted_pizzas'] = models['pizza_linear'].predict(X_future)
    
    future_df['predicted_revenue'] = models['revenue_linear'].predict(X_future)
    
    return future_df[['date', 'weekday', 'predicted_pizzas', 'predicted_revenue']]

def predict_ingredient_demand(predicted_pizzas, pizza_mix_dict, recipes_dict):
    """Convert predicted pizza demand to ingredient demand"""
    ingredient_demand = defaultdict(int)
    
    for pizza_name, proportion in pizza_mix_dict.items():
        if pizza_name in recipes_dict:
            expected_qty = predicted_pizzas * proportion
            for ingredient in recipes_dict[pizza_name]:
                ingredient_demand[ingredient] += expected_qty
    
    return dict(ingredient_demand)

print("✅ Prediction functions ready!")

✅ Prediction functions ready!


# 🔮 Interactive Daily Prediction Tool

## Enter any future date below and get detailed predictions!

This section allows you to input any date and get:
- Total pizzas expected
- Which pizza types to prepare 
- Exact ingredient quantities needed
- Staffing and preparation recommendations

In [ ]:
# Cell 7 — Purpose and rationale
# Purpose: Provide a callable interface to forecast demand for arbitrary target dates. Translate forecasted demand into bill‑of‑materials quantities via recipe composition.
# Inputs: Accepts function arguments as declared; ensure types align with usage (e.g., pandas Timestamp, ndarray).
# Outputs: Returns computed objects explicitly from function(s). Emits textual summaries to stdout for quick verification.
# Key operations:
#  • Function builds features for provided date(s) and returns point forecasts.
#  • Map forecasted pizza counts to ingredient quantities via per‑pizza recipe weights.
# Assumptions/Caveats: Review intermediate outputs to ensure data and features conform to expectations.
# --- End of documentation block ---

def predict_single_day(target_date, model_type='linear'):
    """
    🎯 INTERACTIVE DAILY PREDICTION TOOL
    
    Predict detailed sales information for a specific date
    
    Parameters:
    target_date: str - Date in format 'YYYY-MM-DD' or 'MM-DD-YYYY'
    model_type: str - 'linear' or 'polynomial'
    
    Returns:
    Detailed prediction report
    """
    try:
        # Parse the input date
        if isinstance(target_date, str):
            # Try different date formats
            for date_format in ['%Y-%m-%d', '%m-%d-%Y', '%d-%m-%Y', '%Y/%m/%d', '%m/%d/%Y', '%d/%m/%Y']:
                try:
                    parsed_date = datetime.strptime(target_date, date_format)
                    break
                except ValueError:
                    continue
            else:
                raise ValueError("Date format not recognized. Please use YYYY-MM-DD, MM-DD-YYYY, or similar formats")
        else:
            parsed_date = target_date
        
        # Get basic prediction
        prediction_df = predict_future_demand(parsed_date, parsed_date, model_type)
        
        if len(prediction_df) == 0:
            return "No prediction available for this date"
        
        # Extract single day data
        day_data = prediction_df.iloc[0]
        predicted_pizzas = day_data['predicted_pizzas']
        predicted_revenue = day_data['predicted_revenue']
        weekday = day_data['weekday']
        
        # Calculate pizza type distribution based on historical patterns
        pizza_types_prediction = {}
        
        for pizza_name, proportion in pizza_mix.items():
            expected_qty = predicted_pizzas * proportion
            if expected_qty >= 0.5:  # Only show pizzas with meaningful quantity
                pizza_types_prediction[pizza_name] = expected_qty
        
        # Sort by quantity
        sorted_pizzas = sorted(pizza_types_prediction.items(), key=lambda x: x[1], reverse=True)
        
        # Calculate ingredient requirements
        ingredient_requirements = predict_ingredient_demand(
            predicted_pizzas, 
            pizza_mix.to_dict(), 
            recipes_dict
        )
        
        # Sort ingredients by quantity
        sorted_ingredients = sorted(ingredient_requirements.items(), key=lambda x: x[1], reverse=True)
        
        # Generate report
        print("🍕" * 50)
        print(f"📅 DAILY PREDICTION FOR {parsed_date.strftime('%A, %B %d, %Y')}")
        print("🍕" * 50)
        
        print(f"\n📊 OVERALL FORECAST:")
        print(f"  🍕 Total Pizzas Expected: {predicted_pizzas:.0f} pizzas")
        print(f"  💰 Expected Revenue: €{predicted_revenue:.2f}")
        print(f"  📅 Day of Week: {weekday}")
        print(f"  💡 Model Used: {model_type.title()}")
        
        # Provide context
        avg_daily = daily_sales['pizzas_sold'].mean()
        performance = "Above Average" if predicted_pizzas > avg_daily else "Below Average" if predicted_pizzas < avg_daily * 0.9 else "Average"
        difference = ((predicted_pizzas / avg_daily - 1) * 100)
        print(f"  📈 Performance: {performance} ({difference:+.1f}% vs daily average)")
        
        print(f"\n🍕 TOP PIZZA TYPES TO PREPARE ({len([p for p in sorted_pizzas if p[1] >= 1])} varieties):")
        count = 0
        for pizza_name, qty in sorted_pizzas:
            if qty >= 1 and count < 10:  # Show top 10 pizzas with qty >= 1
                print(f"  • {pizza_name}: {qty:.0f} pizzas")
                count += 1
        
        print(f"\n🥕 INGREDIENT REQUIREMENTS (Top 15):")
        for i, (ingredient, qty) in enumerate(sorted_ingredients[:15]):
            if qty >= 1:
                print(f"  • {ingredient}: {qty:.0f} units")
        
        print(f"\n🎯 PREPARATION RECOMMENDATIONS:")
        
        # Weekend vs weekday advice
        if weekday in ['Saturday', 'Sunday']:
            print(f"  • 📅 Weekend day - expect higher demand than weekdays")
            print(f"  • 👥 Consider extra staffing")
        else:
            print(f"  • 📅 Weekday - standard staffing levels should suffice")
        
        # Top ingredients advice
        critical_ingredients = sorted_ingredients[:3]
        print(f"  • 🚨 CRITICAL: Ensure sufficient stock of:")
        for ingredient, qty in critical_ingredients:
            print(f"    - {ingredient}: {qty:.0f} units")
        
        # Revenue advice
        avg_revenue = daily_sales['revenue'].mean()
        if predicted_revenue > avg_revenue * 1.1:
            print(f"  • 💰 High revenue day expected - prepare for busy service")
        elif predicted_revenue < avg_revenue * 0.9:
            print(f"  • 💰 Lower revenue day - good time for prep work or promotions")
        
        print("\n" + "🍕" * 50)
        
        return {
            'date': parsed_date,
            'predicted_pizzas': predicted_pizzas,
            'predicted_revenue': predicted_revenue,
            'pizza_types': dict(sorted_pizzas),
            'ingredients': dict(sorted_ingredients),
            'weekday': weekday
        }
        
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        print("Please use date format like: '2024-03-15' or '03-15-2024'")
        return None

print("✅ Interactive prediction tool ready!")
print("Use predict_single_day('YYYY-MM-DD') to get predictions for any date")

✅ Interactive prediction tool ready!
Use predict_single_day('YYYY-MM-DD') to get predictions for any date


## 🎯 EASY DATE INPUT SECTION
### Simply change the date below and run the cell to get predictions!

In [ ]:
# Cell 8 — Purpose and rationale
# Purpose: Support the overall data preparation, analysis, or reporting workflow.
# Inputs: No external inputs beyond variables defined in earlier cells.
# Outputs: Emits textual summaries to stdout for quick verification.
# Key operations:
# Assumptions/Caveats: Review intermediate outputs to ensure data and features conform to expectations.
# --- End of documentation block ---

# 🔄 CHANGE THIS DATE TO GET PREDICTIONS FOR ANY DAY:
target_prediction_date = "2024-10-15"  # ⬅️ CHANGE THIS DATE!

print("🎯 INTERACTIVE PREDICTION TOOL")
print("=" * 50)
print(f"Making prediction for: {target_prediction_date}")
print("(To change date, modify the 'target_prediction_date' variable above)")
print("=" * 50)

# Make the prediction
result = predict_single_day(target_prediction_date, model_type='linear')

🎯 INTERACTIVE PREDICTION TOOL
Making prediction for: 2024-10-15
(To change date, modify the 'target_prediction_date' variable above)
🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕
📅 DAILY PREDICTION FOR Tuesday, October 15, 2024
🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕

📊 OVERALL FORECAST:
  🍕 Total Pizzas Expected: 133 pizzas
  💰 Expected Revenue: €2190.12
  📅 Day of Week: Tuesday
  💡 Model Used: Linear
  📈 Performance: Average (-4.2% vs daily average)

🍕 TOP PIZZA TYPES TO PREPARE (32 varieties):
  • The Classic Deluxe Pizza: 7 pizzas
  • The Barbecue Chicken Pizza: 7 pizzas
  • The Hawaiian Pizza: 6 pizzas
  • The Pepperoni Pizza: 6 pizzas
  • The Thai Chicken Pizza: 6 pizzas
  • The California Chicken Pizza: 6 pizzas
  • The Sicilian Pizza: 5 pizzas
  • The Spicy Italian Pizza: 5 pizzas
  • The Southwest Chicken Pizza: 5 pizzas
  • The Big Meat Pizza: 5 pizzas

🥕 INGREDIENT REQUIREMENTS (Top 15):
  • Garlic: 75 units
  • Tomatoes: 72 units
  • Red Onions: 53 units
  •

## 🚀 Multiple Date Predictions
### Try these example dates or add your own!

In [ ]:
# Cell 9 — Purpose and rationale
# Purpose: Translate forecasted demand into bill‑of‑materials quantities via recipe composition.
# Inputs: No external inputs beyond variables defined in earlier cells.
# Outputs: Emits textual summaries to stdout for quick verification.
# Key operations:
#  • Map forecasted pizza counts to ingredient quantities via per‑pizza recipe weights.
# Assumptions/Caveats: Review intermediate outputs to ensure data and features conform to expectations.
# --- End of documentation block ---

print("🍕 PIZZA DEMAND FORECASTING - BATCH PREDICTIONS")
print("=" * 60)

# 🔄 CHANGE THESE DATES TO GET PREDICTIONS FOR ANY DAY:
dates_to_predict = [
    "2024-07-04",     # Independence Day
    "2024-12-25",     # Christmas  
    "2024-06-15",     # Random Saturday
    "2024-03-18",     # Random Monday
    "2024-10-31"      # Halloween
]

print(f"🔮 Making predictions for {len(dates_to_predict)} dates...\n")

# Get predictions for all dates
all_predictions = {}
for date in dates_to_predict:
    print(f"📅 Predicting for {date}:")
    print("-" * 30)
    result = predict_single_day(date, model_type='linear')
    if result:
        all_predictions[date] = result
    print("\n" + "="*60 + "\n")

print("✅ All predictions complete!")
print("\n💡 HOW TO USE THIS TOOL:")
print("  1. Modify the dates in the 'dates_to_predict' list above")
print("  2. Re-run this cell to get new predictions")
print("  3. Use any date format: 'YYYY-MM-DD', 'MM-DD-YYYY', etc.")
print("  4. The tool will tell you:")
print("     • Total pizzas expected")
print("     • Which pizza types to prepare") 
print("     • Exact ingredient quantities needed")
print("     • Staffing and preparation recommendations")

🍕 PIZZA DEMAND FORECASTING - BATCH PREDICTIONS
🔮 Making predictions for 5 dates...

📅 Predicting for 2024-07-04:
------------------------------
🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕
📅 DAILY PREDICTION FOR Thursday, July 04, 2024
🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕🍕

📊 OVERALL FORECAST:
  🍕 Total Pizzas Expected: 152 pizzas
  💰 Expected Revenue: €2511.77
  📅 Day of Week: Thursday
  💡 Model Used: Linear
  📈 Performance: Above Average (+9.9% vs daily average)

🍕 TOP PIZZA TYPES TO PREPARE (32 varieties):
  • The Classic Deluxe Pizza: 8 pizzas
  • The Barbecue Chicken Pizza: 7 pizzas
  • The Hawaiian Pizza: 7 pizzas
  • The Pepperoni Pizza: 7 pizzas
  • The Thai Chicken Pizza: 7 pizzas
  • The California Chicken Pizza: 7 pizzas
  • The Sicilian Pizza: 6 pizzas
  • The Spicy Italian Pizza: 6 pizzas
  • The Southwest Chicken Pizza: 6 pizzas
  • The Big Meat Pizza: 6 pizzas

🥕 INGREDIENT REQUIREMENTS (Top 15):
  • Garlic: 86 units
  • Tomatoes: 83 units
  • Red Onio